%pip install ollama

import ollama

res = ollama.chat(model='llama3-ko', messages=[
    {
        'role': 'user',
        'content': '코사인의 미분 공식에 대해 설명해줘'
    }
])
print(res['message']['content'])

In [30]:
from flask_socketio import SocketIO, emit, join_room, leave_room
from flask import *
from flask_login import *
from flask_session import Session
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
# from datetime import timedelta
from langchain.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.prompts.prompt import PromptTemplate
import torch

llm_model = ChatOllama(
    model='meta-llama-3.1',
    num_predict=256,
    format='json'
)

device = 'cpu'

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': device}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)


prompt_template = '''Use the following pieces of context to answer the question at the end.
If you don't find the answer in context, just say that '내용을 확인할 수 없습니다.', don't try to make up an answer.
If you find the answer in context, answer me only use korean.

context: {context}

Question: {question}
Helpful Answer:'''
rag_prompt = PromptTemplate.from_template(prompt_template)

vectorstore = FAISS.load_local(
    'test',
    embedding_model,
    allow_dangerous_deserialization=True,
)
# params = request.get_json()
# question = params['question']

memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True,
)

In [33]:
question = '`content`: list [보고서의 목차] resonse in JSON format.'

memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True,
)

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm_model,
    retriever=vectorstore.as_retriever(),
    condense_question_prompt=rag_prompt,
    memory=memory,
)
res = conversation_chain({'question': question})
# print(res['chat_history'][1].content)

print(
    res['chat_history'][1].content
)

{ "content": ["1. Google AIStudio를검색한후아래사이트로접속및로그인-> ‘Gemini API 키가져오기’ 클릭", "2. 로그인후‘Get API Key’ 클릭", "3. ‘API 키만들기’ 클릭", "4. ‘새프로젝트에서API 키만들기’ 버튼클릭", "5. API Key 생성된정보확인", "6. 해당페이지에서API 키는재확인및복사가능"] } 	                    		                    		                    		                    		                    		                    					                    					                    					                    					                    					                    					                    					                    					                    					                    


In [34]:
aa = { "content": ["1. Google AIStudio를검색한후아래사이트로접속및로그인-> ‘Gemini API 키가져오기’ 클릭", "2. 로그인후‘Get API Key’ 클릭", "3. ‘API 키만들기’ 클릭", "4. ‘새프로젝트에서API 키만들기’ 버튼클릭", "5. API Key 생성된정보확인", "6. 해당페이지에서API 키는재확인및복사가능"] }
aa

{'content': ['1. Google AIStudio를검색한후아래사이트로접속및로그인-> ‘Gemini API 키가져오기’ 클릭',
  '2. 로그인후‘Get API Key’ 클릭',
  '3. ‘API 키만들기’ 클릭',
  '4. ‘새프로젝트에서API 키만들기’ 버튼클릭',
  '5. API Key 생성된정보확인',
  '6. 해당페이지에서API 키는재확인및복사가능']}

In [32]:
#pip install psycopg2
import psycopg2
from psycopg2 import pool

db = psycopg2.connect(
    user='postgres.vpcdvbdktvvzrvjfyyzm',
    password='Odvv8E1iChKjwai4',
    host='aws-0-ap-southeast-1.pooler.supabase.com',
    port=6543,
    dbname='postgres'
)